In [1]:
import QUANTAXIS as qa
import pandas as pd
import time
import datetime

提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改


In [2]:
df=pd.read_excel("E:\\leo\\投资\\2021\\股票持仓分析20210622-快照20210622.xlsx",sheet_name="持仓明细详情",header=2)
df=df[df['pos_num']>0]
code_cn_list=list(x.split(".")[0] for x in df.code[df['currency']=='CNY'].unique().tolist())
code_hk_list=list(x.split(".")[0] for x in df.code[df['currency']=='HK'].unique().tolist())
df_cn=df[df['currency']=='CNY'].assign(code=df['code'].apply(lambda x:x.split(".")[0]))

def QA_fetch_fund_name(code, collections=qa.DATABASE.fund_list):
    """
    获取股票名称
    """
    if isinstance(code, str):
        try:
            res = collections.find_one({'code': code})
            return res['name']
        except Exception as e:
            if (res is None):
                return qa.QA_fetch_etf_name(code)
            return code
def cn_mkt():
    real_price=qa.QA_fetch_get_stock_realtime("tdx",code=code_cn_list).reset_index(drop=False)
    real_price=real_price.assign(r=((real_price['price']/real_price['last_close']-1)*100).round(2))
    real_price=real_price.assign(last_close=real_price.apply(lambda x:x['last_close'] if x['code'][0] in ['0','3','6'] else x['last_close']/10,axis=1),
                             price=real_price.apply(lambda x:x['price'] if x['code'][0] in ['0','3','6'] else x['price']/10,axis=1))
                             
    real_price=real_price.assign(name=real_price['code'].apply(lambda x:qa.QA_fetch_stock_name(x) if x[0] in ['0','3','6'] else QA_fetch_fund_name(x)),pos=real_price['code'].apply(lambda x:df_cn[df_cn['code']==x]['pos_num'].sum()))
    real_price=real_price.assign(mv=real_price['price']*real_price['pos'],last_mv=real_price['last_close']*real_price['pos'])

    real=real_price[['code','name','last_close','price','r','pos','last_mv','mv']].assign(pofit=(real_price['pos']*real_price['last_close']*real_price['r']/100).round(2),w=(real_price['mv']/real_price['mv'].sum()*100).round(2)).sort_values('r',ascending=False)
    print('total:%s %s %s'% (real['mv'].sum().round(2),str((real['pofit'].sum()/real['last_mv'].sum()).round(4)*100)+'%',real['pofit'].sum().round(2)))
    return real
def clac_max_min(end='2021-06-05'):
#     end='2021-06-05'
    ddf_stock=qa.QA_fetch_stock_day_adv(code_cn_list1,start='2021-01-01',end=end).data
    ddf_index=qa.QA_fetch_index_day_adv(code_cn_list1,start='2021-01-01',end=end).data
    ddf=ddf_stock.append(ddf_index).reset_index(drop=False) #to_qfq().
    def max_min(x):
        idmax=x['close'].idxmax()
        idmin=x.loc[idmax:]['close'].idxmin()
        return pd.Series(index=['max','maxdate','min','mindate'],data=[x.loc[idmax]['close'],x.loc[idmax]['date'],x.loc[idmin]['close'],x.loc[idmin]['date']])
    data=ddf.groupby('code').apply(lambda x:max_min(x))
    ddf2=ddf.groupby('code').last().reset_index(drop=False)
    ddf2=ddf2.merge(data,on='code')
    ddf2['name']=ddf2['code'].apply(lambda x:qa.QA_fetch_stock_name(x) if x[0] in ['0','3','6'] else QA_fetch_fund_name(x))
    ddf2=ddf2.assign(r=(ddf2['close']/ddf2['max']*100-100).round(2),f=(ddf2['close']/ddf2['min']*100-100).round(2),down=(ddf2['min']/ddf2['max']*100-100).round(2)).sort_values('r')
    return ddf2[['code','date','name','close','max','maxdate','min','mindate','r','f','down']]
def get_hk_quote():
    end=time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    start=datetime.datetime.strptime(end, "%Y-%m-%d %H:%M:%S")-datetime.timedelta(minutes=15)
    hk_codes=['01088', '01516', '01918', '02007', '00700']
    for code in hk_codes:
        print(qa.QA_fetch_get_hkstock_min('tdx',code=code,start=start,end=end)['close'][0])

In [4]:
cn_mkt()

total:1841682.3 2.65% 47489.05


,code,name,last_close,price,r,pos,last_mv,mv,pofit,w
12,600985,淮北矿业,12.330,13.560,9.98,16024,197575.920,217285.440,19718.08,11.80
9,502020,50增强,1.230,1.280,4.07,720,885.600,921.600,36.04,0.05
2,601225,陕西煤业,12.090,12.510,3.47,15500,187395.000,193905.000,6502.61,10.53
0,600036,招商银行,55.000,56.400,2.55,3500,192500.000,197400.000,4908.75,10.72
6,002027,分众传媒,8.950,9.150,2.23,20200,180790.000,184830.000,4031.62,10.04
10,600516,方大炭素,7.720,7.840,1.55,1043,8051.960,8177.120,124.81,0.44
1,601166,兴业银行,20.440,20.720,1.37,39700,811468.000,822584.000,11117.11,44.66
5,160631,银行指基,1.033,1.046,1.26,2447,2527.751,2559.562,31.85,0.14
3,601318,中国平安,65.960,66.680,1.09,900,59364.000,60012.000,647.07,3.26
11,601939,建设银行,6.670,6.740,1.05,3600,24012.000,24264.000,252.13,1.32
